In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110670

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,36304.5678
2018-02-28,35200.2033
2018-03-31,45773.6656
2018-04-30,57631.7766
2018-05-31,59670.2664
2018-06-30,62321.4245
2018-07-31,65656.7906
2018-08-31,61165.0252
2018-09-30,86757.6388


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    36304.5678
 2018-02-28    35200.2033
 2018-03-31    45773.6656
 2018-04-30    57631.7766
 2018-05-31    59670.2664
 2018-06-30    62321.4245
 2018-07-31    65656.7906
 2018-08-31    61165.0252
 2018-09-30    86757.6388
 2018-10-31    58374.8990
 2018-11-30    52909.8008
 2018-12-31    61591.2492
 2019-01-31    68966.6256
 2019-02-28    66231.4472
 2019-03-31    69955.7868
 2019-04-30    54352.8522
 2019-05-31    67148.2431
 2019-06-30    74292.8533
 2019-07-31    78348.7495
 2019-08-31    70246.9059
 2019-09-30    67831.7799
 2019-10-31    71242.5661
 2019-11-30    59287.3389
 2019-12-31    59162.8192
 2020-01-31    58301.1136
 2020-02-29    59719.1901
 2020-03-31    63690.5429
 2020-04-30    60883.7775
 2020-05-31    65781.5868
 2020-06-30    70401.0084
 2020-07-31    75752.8286
 2020-08-31    69328.3142
 2020-09-30    64135.6259
 2020-10-31    62397.2926
 2020-11-30    65943.3490
 2020-12-31    65118.1515
 2021-01-31    73613.1067
 2021-02-

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.265516
p-value : 0.016492
Critical Values :
	1%: -3.632743
	5%: -2.948510
	10%: -2.613017


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=897.011, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=892.766, Time=0.21 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=895.562, Time=0.06 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=903.926, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=895.661, Time=0.29 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.39 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=890.747, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=897.383, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=896.428, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=904.667, Time=0.11 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=909.927, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=896.397, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=inf, Time=0.29 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=896.823, Time=0.27 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=903.557, Time=0.36 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -441.767
Method:                       css-mle   S.D. of innovations           8919.753
Date:                Sat, 11 Sep 2021   AIC                            889.534
Time:                        04:19:31   BIC                            894.747
Sample:                             0   HQIC                           891.444
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        6.32e+04   2679.303     23.588      0.000    5.79e+04    6.85e+04
ar.L1.y        0.4917      0.149      3.297      0.001       0.199       0.784
                                    Roots           

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([61054.49669807, 62144.90859943]), array([8919.75322304, 9939.60947969]), array([[43572.10162992, 78536.89176621],
       [42663.63199885, 81626.18520002]]))
